# Home 4: Build a seq2seq model for machine translation.

### Name: [Your-Name?]

### Task: Translate English to [what-language?]

## 0. You will do the following:

1. Read and run my code.
2. Complete the code in Section 1.1 and Section 4.2.

    * Translation English to **German** is not acceptable!!! Try another language.
    
3. **Make improvements.** Directly modify the code in Section 3. Do at least one of the followings. By doing more, you will get up to 2 bonus scores to the total.

    * Bi-LSTM instead of LSTM
    
    * Multi-task learning (e.g., both English to French and English to Spanish)
    
    * Attention
    
4. Evaluate the translation using the BLEU score. 

    * Optional. Up to 2 bonus scores to the total.
    
5. Convert the notebook to .HTML file. 

    * The HTML file must contain the code and the output after execution.

6. Put the .HTML file in your own Github repo. 

7. Submit the link to the HTML file to Canvas

    * E.g., https://github.com/wangshusen/CS583A-2019Spring/blob/master/homework/HM4/seq2seq.html

#### Hint: To implement Bi-LSTM, you will need the following code to build the encoder; the decoder won't be much different.

In [0]:
from keras.layers import Bidirectional, Concatenate

encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

## 1. Data preparation

1. Download data (e.g., "deu-eng.zip") from http://www.manythings.org/anki/
2. Unzip the .ZIP file.
3. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".

### 1.1. Load and clean text


In [0]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [0]:
# e.g., filename = 'Data/deu.txt'
filename_fra = 'fra.txt'
filename_ita = 'ita.txt'
# e.g., n_train = 20000
n_train = 4000

In [0]:
# load dataset
doc_fra = load_doc(filename_fra)
doc_ita = load_doc(filename_ita)

# split into Language1-Language2 pairs
pairs_fra = to_pairs(doc_fra)
pairs_ita = to_pairs(doc_ita)

# clean sentences
clean_pairs_fra = clean_data(pairs_fra)[0:n_train, :]
clean_pairs_ita = clean_data(pairs_ita)[0:n_train, :]

In [4]:
clean_pairs_ita

array([['hi', 'ciao'],
       ['run', 'corri'],
       ['run', 'corra'],
       ...,
       ['are you new', 'siete nuove'],
       ['are you new', 'e nuovo'],
       ['are you new', 'e nuova']], dtype='<U313')

In [5]:
for i in range(3000, 3010):
    print('[' + clean_pairs_ita[i, 0] + '] => [' + clean_pairs_ita[i, 1] + ']')

[im flabby] => [io sono fiacco]
[im flabby] => [sono fiacca]
[im flabby] => [io sono fiacca]
[im for it] => [sono a favore]
[im for it] => [io sono a favore]
[im frugal] => [sono parsimonioso]
[im greedy] => [sono avido]
[im greedy] => [io sono avido]
[im greedy] => [sono avida]
[im greedy] => [io sono avida]


In [6]:
input_texts_fra = clean_pairs_fra[:, 0]
target_texts_fra = ['\t' + text + '\n' for text in clean_pairs_fra[:, 1]]

print('Length of input_texts_fra:  ' + str(input_texts_fra.shape))
#print('Length of target_texts_fra: ' + str(target_texts_fra.shape))


input_texts_ita = clean_pairs_ita[:, 0]
target_texts_ita = ['\t' + text + '\n' for text in clean_pairs_ita[:, 1]]
print('==========================================')
print('Length of input_texts_ita:  ' + str(input_texts_ita.shape))
#print('Length of target_texts_ita: ' + str(target_texts_ita.shape))

Length of input_texts_fra:  (4000,)
Length of input_texts_ita:  (4000,)


In [7]:
max_encoder_seq_length_fra = max(len(line) for line in input_texts_fra)
max_decoder_seq_length_fra = max(len(line) for line in target_texts_fra)

print('max length of fra input  sentences: %d' % (max_encoder_seq_length_fra))
print('max length of fra target sentences: %d' % (max_decoder_seq_length_fra))

max length of fra input  sentences: 12
max length of fra target sentences: 56


In [8]:
max_encoder_seq_length_ita = max(len(line) for line in input_texts_ita)
max_decoder_seq_length_ita = max(len(line) for line in target_texts_ita)

print('max length of ita input  sentences: %d' % (max_encoder_seq_length_ita))
print('max length of ita target sentences: %d' % (max_decoder_seq_length_ita))

max length of ita input  sentences: 11
max length of ita target sentences: 34


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq_fra, input_token_index_fra = text2sequences(max_encoder_seq_length_fra, 
                                                      input_texts_fra)
decoder_input_seq_fra, target_token_index_fra = text2sequences(max_decoder_seq_length_fra, 
                                                       target_texts_fra)


encoder_input_seq_ita, input_token_index_ita = text2sequences(max_encoder_seq_length_ita, 
                                                      input_texts_ita)
decoder_input_seq_ita, target_token_index_ita = text2sequences(max_decoder_seq_length_ita, 
                                                       target_texts_ita)
print('shape of encoder_input_seq_fra: ' + str(encoder_input_seq_fra.shape))
print('shape of input_token_index_fra: ' + str(len(input_token_index_fra)))
print('shape of decoder_input_seq_fra: ' + str(decoder_input_seq_fra.shape))
print('shape of target_token_index_fra: ' + str(len(target_token_index_fra)))


print('====================================')
print('shape of encoder_input_seq_ita: ' + str(encoder_input_seq_ita.shape))
print('shape of input_token_index_ita: ' + str(len(input_token_index_ita)))
print('shape of decoder_input_seq_ita: ' + str(decoder_input_seq_ita.shape))
print('shape of target_token_index_ita: ' + str(len(target_token_index_ita)))

Using TensorFlow backend.


shape of encoder_input_seq_fra: (4000, 12)
shape of input_token_index_fra: 27
shape of decoder_input_seq_fra: (4000, 56)
shape of target_token_index_fra: 28
shape of encoder_input_seq_ita: (4000, 11)
shape of input_token_index_ita: 27
shape of decoder_input_seq_ita: (4000, 34)
shape of target_token_index_ita: 28


In [11]:
encoder_input_seq_ita.shape

(4000, 11)

In [12]:
num_encoder_tokens_fra = len(input_token_index_fra) + 1
num_decoder_tokens_fra = len(target_token_index_fra) + 1

print('num_encoder_tokens_fra: ' + str(num_encoder_tokens_fra))
print('num_decoder_tokens_fra: ' + str(num_decoder_tokens_fra))


num_encoder_tokens_ita = len(input_token_index_ita) + 1
num_decoder_tokens_ita = len(target_token_index_ita) + 1

print('num_encoder_tokens_ita: ' + str(num_encoder_tokens_ita))
print('num_decoder_tokens_ita: ' + str(num_decoder_tokens_ita))

num_encoder_tokens_fra: 28
num_decoder_tokens_fra: 29
num_encoder_tokens_ita: 28
num_decoder_tokens_ita: 29


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [13]:
target_texts_ita[100]

'\tlavoro ai ferri\n'

In [14]:
decoder_input_seq_ita[100, :]

array([ 5, 13,  3, 18,  1, 11,  1,  2,  3,  4,  2, 21,  7, 11, 11,  4,  6,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [15]:
from keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data_fra = onehot_encode(encoder_input_seq_fra, max_encoder_seq_length_fra, num_encoder_tokens_fra)
decoder_input_data_fra = onehot_encode(decoder_input_seq_fra, max_decoder_seq_length_fra, num_decoder_tokens_fra)

decoder_target_seq_fra = numpy.zeros(decoder_input_seq_fra.shape)
decoder_target_seq_fra[:, 0:-1] = decoder_input_seq_fra[:, 1:]
decoder_target_data_fra = onehot_encode(decoder_target_seq_fra, 
                                    max_decoder_seq_length_fra, 
                                    num_decoder_tokens_fra)



encoder_input_data_ita = onehot_encode(encoder_input_seq_ita, max_encoder_seq_length_ita, num_encoder_tokens_ita)
decoder_input_data_ita = onehot_encode(decoder_input_seq_ita, max_decoder_seq_length_ita, num_decoder_tokens_ita)

decoder_target_seq_ita = numpy.zeros(decoder_input_seq_ita.shape)
decoder_target_seq_ita[:, 0:-1] = decoder_input_seq_ita[:, 1:]
decoder_target_data_ita = onehot_encode(decoder_target_seq_ita, 
                                    max_decoder_seq_length_ita, 
                                    num_decoder_tokens_ita)
print(encoder_input_data_fra.shape)
print(decoder_input_data_fra.shape)
print('=====================================')
print(encoder_input_data_ita.shape)
print(decoder_input_data_ita.shape)

(4000, 12, 28)
(4000, 56, 29)
(4000, 11, 28)
(4000, 34, 29)


## 3. Build the networks (for training)

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.

### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [0]:
from keras.layers import Input, LSTM
from keras.models import Model
from keras.layers import Bidirectional, Concatenate

latent_dim = 256
def encoder(num_encoder_tokens):
    # inputs of the encoder network
    encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                           name='encoder_inputs')

    # set the BI-LSTM layer
    '''encoder_lstm = LSTM(latent_dim, return_state=True, 
                       dropout=0.5, name='encoder_lstm')
    _, state_h, state_c = encoder_lstm(encoder_inputs)'''
    encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                      dropout=0.5, name='encoder_lstm'))
    _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])

    # build the encoder network model
    encoder_model = Model(inputs=encoder_inputs, 
                          outputs=[state_h, state_c],
                          name='encoder')
    return encoder_model


In [0]:
encoder_model_fra = encoder(num_encoder_tokens_fra)
encoder_model_ita = encoder(num_encoder_tokens_ita)

Print a summary and save the encoder network structure to "./encoder.pdf"

In [0]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

def plot_encoder_summary(encoder_model):
    SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

    plot_model(
        model=encoder_model, show_shapes=False,
        to_file='encoder.pdf'
    )

    encoder_model.summary()

In [33]:
print('===============')
plot_encoder_summary(encoder_model_fra)

print('===============')
plot_encoder_summary(encoder_model_ita)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 28)     0                                            
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) [(None, 512), (None, 583680      encoder_inputs[0][0]             
__________________________________________________________________________________________________
concatenate_9 (Concatenate)     (None, 512)          0           bidirectional_5[0][1]            
                                                                 bidirectional_5[0][3]            
__________________________________________________________________________________________________
concatenate_10 (Concatenate)    (None, 512)          0           bidirectional_5[0][2]            
          

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [0]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# inputs of the decoder network
decoder_input_h_fra = Input(shape=(latent_dim*2,), name='decoder_input_h')
decoder_input_c_fra = Input(shape=(latent_dim*2,), name='decoder_input_c')
decoder_input_x_fra = Input(shape=(None, num_decoder_tokens_fra), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, 
                        return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs_fra, state_h_fra, state_c_fra = decoder_lstm(decoder_input_x_fra, 
                                                          initial_state=[decoder_input_h_fra, decoder_input_c_fra])

# set the dense layer
decoder_dense_fra = Dense(num_decoder_tokens_fra, activation='softmax', name='decoder_dense_fra')
decoder_outputs_fra = decoder_dense_fra(decoder_lstm_outputs_fra)

# build the decoder network model
decoder_model_fra = Model(inputs=[decoder_input_x_fra, decoder_input_h_fra, decoder_input_c_fra],
                          outputs=[decoder_outputs_fra, state_h_fra, state_c_fra],
                          name='decoder_fra')
    


In [0]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# inputs of the decoder network
decoder_input_h_ita = Input(shape=(latent_dim*2,), name='decoder_input_h')
decoder_input_c_ita = Input(shape=(latent_dim*2,), name='decoder_input_c')
decoder_input_x_ita = Input(shape=(None, num_decoder_tokens_ita), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, 
                        return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs_ita, state_h_ita, state_c_ita = decoder_lstm(decoder_input_x_ita, 
                                                          initial_state=[decoder_input_h_ita, decoder_input_c_ita])

# set the dense layer
decoder_dense_ita = Dense(num_decoder_tokens_ita, activation='softmax', name='decoder_dense_ita')
decoder_outputs_ita = decoder_dense_ita(decoder_lstm_outputs_ita)

# build the decoder network model
decoder_model_ita = Model(inputs=[decoder_input_x_ita, decoder_input_h_ita, decoder_input_c_ita],
                          outputs=[decoder_outputs_ita, state_h_ita, state_c_ita],
                          name='decoder_ita')
    


Print a summary and save the encoder network structure to "./decoder.pdf"

In [0]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

def plot_decoder_model(decoder_model):
    SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

    plot_model(
        model=decoder_model, show_shapes=False,
        to_file='decoder.pdf'
    )

    decoder_model.summary()

In [67]:
plot_decoder_model(decoder_model_fra)
print('================================')
plot_decoder_model(decoder_model_ita)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input_x (InputLayer)    (None, None, 29)     0                                            
__________________________________________________________________________________________________
decoder_input_h (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_input_c (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1110016     decoder_input_x[0][0]            
                                                                 decoder_input_h[0][0]            
          

### 3.3. Connect the encoder and decoder

In [0]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens_fra), name='encoder_input_x_fra')
decoder_input_x_fra = Input(shape=(None, num_decoder_tokens_fra), name='decoder_input_x_fra')
decoder_input_x_ita = Input(shape=(None, num_decoder_tokens_ita), name='decoder_input_x_ita')


# connect encoder to decoder
encoder_final_states = encoder_model_fra([encoder_input_x])
decoder_lstm_output_fra, _, _ = decoder_lstm(decoder_input_x_fra, initial_state=encoder_final_states)
decoder_pred_fra = decoder_dense_fra(decoder_lstm_output_fra)

decoder_lstm_output_ita, _, _ = decoder_lstm(decoder_input_x_ita, initial_state=encoder_final_states)
decoder_pred_ita = decoder_dense_ita(decoder_lstm_output_ita)

model = Model(inputs=[encoder_input_x, decoder_input_x_fra, decoder_input_x_ita], 
              outputs=[decoder_pred_fra,decoder_pred_ita], 
              name='model_training')

In [65]:
print(state_h_ita)
print(decoder_input_h_ita)

Tensor("decoder_lstm_11/while/Exit_2:0", shape=(?, 512), dtype=float32)
Tensor("decoder_input_h_8:0", shape=(?, 512), dtype=float32)


In [69]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_x_fra (InputLayer (None, None, 28)     0                                            
__________________________________________________________________________________________________
decoder_input_x_fra (InputLayer (None, None, 29)     0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 512), (None, 583680      encoder_input_x_fra[0][0]        
__________________________________________________________________________________________________
decoder_input_x_ita (InputLayer (None, None, 29)     0                                            
__________________________________________________________________________________________________
decoder_ls

### 3.5. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [70]:
print('shape of encoder_input_data' + str(encoder_input_data_fra.shape))
print('shape of decoder_input_data' + str(decoder_input_data_fra.shape))
print('shape of decoder_target_data' + str(decoder_target_data_fra.shape))


print('=====================================================')
print('shape of encoder_input_data' + str(encoder_input_data_ita.shape))
print('shape of decoder_input_data' + str(decoder_input_data_ita.shape))
print('shape of decoder_target_data' + str(decoder_target_data_ita.shape))

shape of encoder_input_data(4000, 12, 28)
shape of decoder_input_data(4000, 56, 29)
shape of decoder_target_data(4000, 56, 29)
shape of encoder_input_data(4000, 11, 28)
shape of decoder_input_data(4000, 34, 29)
shape of decoder_target_data(4000, 34, 29)


In [71]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data_fra, decoder_input_data_fra,decoder_input_data_ita],  # training data
          [decoder_target_data_fra,decoder_target_data_ita],                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3200 samples, validate on 800 samples
Epoch 1/50
3200/3200 [==============================] - 14s 4ms/step - loss: 2.6085 - decoder_dense_fra_loss: 1.1384 - decoder_dense_ita_loss: 1.4701 - val_loss: 2.7385 - val_decoder_dense_fra_loss: 1.2221 - val_decoder_dense_ita_loss: 1.5164
Epoch 2/50
3200/3200 [==============================] - 8s 3ms/step - loss: 2.1184 - decoder_dense_fra_loss: 0.9041 - decoder_dense_ita_loss: 1.2143 - val_loss: 2.2366 - val_decoder_dense_fra_loss: 0.9547 - val_decoder_dense_ita_loss: 1.2819
Epoch 3/50
3200/3200 [==============================] - 8s 2ms/step - loss: 1.9176 - decoder_dense_fra_loss: 0.8179 - decoder_dense_ita_loss: 1.0997 - val_loss: 2.0237 - val_decoder_dense_fra_loss: 0.8516 - val_decoder_dense_ita_loss: 1.1721
Epoch 4/50
3200/3200 [==============================] - 8s 3ms/step - loss: 1.7363 - decoder_dense_f

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_7/concatenate_9/concat:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'encoder_7/concatenate_10/concat:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


## 4. Make predictions


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [0]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index_fra.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index_fra.items())

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model_fra.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens_fra))
    target_seq[0, 0, target_token_index_fra['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model_fra.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length_fra):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens_fra))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [84]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data_fra[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts_fra[seq_index])
    print('France (true): ', target_texts_fra[seq_index][1:-1])
    print('France (pred): ', decoded_sentence[0:-1])


-
English:        i buried it
France (true):  je lai enterre
France (pred):  je lai entre
-
English:        i buried it
France (true):  je lai enterree
France (pred):  je lai entre
-
English:        i burned it
France (true):  je lai brule
France (pred):  je lai conte
-
English:        i burned it
France (true):  je lai brulee
France (pred):  je lai conte
-
English:        i came back
France (true):  je suis revenu
France (pred):  je suis detendue
-
English:        i can dance
France (true):  je sais danser
France (pred):  je sais de la duis
-
English:        i can do it
France (true):  je peux le faire
France (pred):  je veux de la me sou
-
English:        i can do it
France (true):  je peux y arriver
France (pred):  je veux de la me sou
-
English:        i can do it
France (true):  je peux y parvenir
France (pred):  je veux de la me sou
-
English:        i can do it
France (true):  jarrive a le faire
France (pred):  je veux de la me sou
-
English:        i cant eat
France (true):  je

### 4.2. Translate an English sentence to the target language

1. Tokenization
2. One-hot encode
3. Translate

In [87]:
input_sentence = 'why is that'

input_x = numpy.zeros((len(input_sentence),max_encoder_seq_length_fra, num_encoder_tokens_fra))

for i , word in enumerate(input_sentence):
  for j, char in enumerate(word):
    word_index = input_token_index_fra.get(char)
    input_x[i,j,word_index] = 1

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + str(input_sentence))
print('translated sentence is: ' + str(translated_sentence))

source sentence is: why is that
translated sentence is: nous sommes en artir



## 5. Evaluate the translation using BLEU score

Reference: 
- https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
- https://en.wikipedia.org/wiki/BLEU


In [88]:
from nltk.translate.bleu_score import sentence_bleu
score = []
for seq_index in range(10, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data_fra[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    score.append(sentence_bleu(target_texts_fra[seq_index][1:-1],decoded_sentence[0:-1]))
print(sum(score)/len(score))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.8224829185437927
